1. Reference Counting (Перша лінія оборони)

Багато розробників думають, що Garbage Collector (GC) — це головний механізм. Це міф. Головний механізм — це Reference Counting (підрахунок посилань). Він працює миттєво і детерміновано.

Як тільки ob_refcnt падає до 0 -> викликається деструктор -> пам'ять звільняється негайно.

🔬 Практика: Доказ миттєвої смерті
Ми використаємо магічний метод __del__.

⚠️ Warning for Production: Ніколи не покладайтеся на __del__ для критичної логіки (закриття файлів/DB), бо в складних циклических посиланнях він може не спрацювати так, як ви очікуєте. Але для навчання — це ідеальний інструмент.

In [1]:
import sys

class DoomObject:
    def __init__(self, name):
        self.name = name
        print(f"{self.name} is born at {hex(id(self))}")

    def __del__(self):
        print(f"{self.name} is destroyed immediately!")

# 1. Створення об'єкта
obj = DoomObject("Victim 1")

# 2. Перевірка посилань (згадуємо: getrefcount додає +1)
print(f"Ref count: {sys.getrefcount(obj) - 1}")

# 3. Видалення посилання
print("Deleting variable...")
del obj
print("End of Cell 1")

Victim 1 is born at 0x287c129c440
Ref count: 1
Deleting variable...
Victim 1 is destroyed immediately!
End of Cell 1


Очікуваний результат: Ви побачите повідомлення "destroyed" до того, як надрукується "End of Cell 1". Це доводить, що очищення відбувається синхронно, без участі GC.

2. Проблема: Reference Cycles (Циклічні посилання)
Reference Counting ідеальний, але має одну фатальну ваду. Якщо два об'єкти посилаються один на одного, їхній refcnt ніколи не стане нулем, навіть якщо ви видалите всі зовнішні змінні.

Це називається Memory Leak (витік пам'яті) в контексті Reference Counting.

In [2]:
import ctypes
import gc

# Очищаємо сміття від попередніх cells, щоб результат був чистим
gc.collect()

# Функція для "брудного" читання реф-каунту з пам'яті (щоб не створювати зайвих посилань)
# ⚠️ Це НЕБЕЗПЕЧНА техніка: якщо об'єкт вже звільнений, ми читаємо сміття.
# Тут це безпечно ТІЛЬКИ тому, що циклічне посилання тримає об'єкти живими.
# Використовуємо c_ssize_t (8 байт на 64-bit), бо ob_refcnt має тип Py_ssize_t.
# ctypes.c_long на Windows лише 4 байти — це дасть некоректний результат для великих refcount!
def get_ref_count(address):
    return ctypes.c_ssize_t.from_address(address).value

# Створюємо два об'єкти
node_a = DoomObject("Node A")
node_b = DoomObject("Node B")

# Отримуємо їх адреси
id_a = id(node_a)
id_b = id(node_b)

# СТВОРЮЄМО ЦИКЛ
print("Linking nodes...")
node_a.partner = node_b
node_b.partner = node_a

# Видаляємо змінні
print("Deleting variables...")
del node_a
del node_b

# Об'єкти живі завдяки циклу — тому читання за адресою безпечне
print(f"Node A ref count (by address): {get_ref_count(id_a)}")
print(f"Node B ref count (by address): {get_ref_count(id_b)}")
print("\nОб'єкти НЕ знищені! refcnt = 1 (партнер тримає посилання)")

Node A is born at 0x287bf97ae90
Node B is born at 0x287bf97ad50
Linking nodes...
Deleting variables...
Node A ref count (by address): 1
Node B ref count (by address): 1

Об'єкти НЕ знищені! refcnt = 1 (партнер тримає посилання)


3. Generational Garbage Collector (GC)
Python GC існує тільки для того, щоб знаходити і вбивати ці цикли. Він не працює постійно (це було б повільно). Він запускається періодично, коли кількість алокацій перевищує поріг.

Теорія поколінь (Generations)
Об'єкти діляться на 3 покоління:

1. Generation 0: Усі новостворені об'єкти потрапляють сюди. GC сканує це покоління найчастіше.
2. Generation 1: Об'єкти, які пережили очищення Gen 0.
3. Generation 2: Об'єкти, які пережили очищення Gen 1. Сюди потрапляють довгоживучі дані (налаштування, глобальні змінні). GC сканує це вкрай рідко.

🔬 Практика: Примусовий виклик GC
Давайте вручну викличемо GC, щоб знищити "зомбі" з попереднього прикладу (Node A і Node B).

In [3]:
import gc

# Перевіримо налаштування порогів (Thresholds)
# Формат: (gen0, gen1, gen2)
# gen0: якщо (allocations - deallocations) > threshold -> запустити GC Gen 0
# gen1: якщо Gen 0 очищався gen1 разів -> запустити GC Gen 1
# Значення за замовчуванням залежать від версії Python:
#   Python ≤ 3.11: (700, 10, 10)
#   Python 3.12+:  (2000, 10, 0) — threshold Gen 0 збільшено для кращої продуктивності
print(f"GC Thresholds: {gc.get_threshold()}")

# Перевіримо статистику (скільки об'єктів у кожному поколінні зараз)
print(f"Objects in generations: {gc.get_count()}")

print("Calling gc.collect() manually...")
# gc.collect() повертає кількість знайдених і знищених об'єктів (unreachable objects)
unreachable_count = gc.collect()

print(f"GC collected {unreachable_count} objects.")
# ТУТ ви повинні побачити повідомлення "destroyed" від Node A та Node B,
# тому що GC знайшов цикл, розірвав його і викликав деструктори.

GC Thresholds: (2000, 10, 0)
Objects in generations: (469, 0, 0)
Calling gc.collect() manually...
Node A is destroyed immediately!
Node B is destroyed immediately!
GC collected 2 objects.


4. Weak References (Слабкі посилання)
Як ви часто будете писати кеші (Cache). Звичайний словник cache = {key: large_object} — це вбивця пам'яті. Поки об'єкт у словнику, його ref count > 0, і він ніколи не видалиться.

Рішення: Weak Reference. Це посилання, яке дозволяє доступ до об'єкта, але не збільшує його Reference Count. Якщо це єдине посилання — об'єкт знищується.

In [4]:
import weakref

class HeavyImage:
    def __init__(self, content):
        self.content = content
    def __repr__(self):
        return f"<Image: {self.content}>"

# 1. Створюємо об'єкт (Ref count = 1)
img = HeavyImage("4K_Texture.png")

# 2. Створюємо слабке посилання
# r() поверне об'єкт, якщо він живий, або None, якщо мертвий
weak_link = weakref.ref(img)

print(f"Weak link calls: {weak_link()}")
print(f"Ref count (approx): {sys.getrefcount(img) - 1}") # Все ще 1, weakref не додав +1

# 3. Видаляємо сильне посилання
print("Deleting strong reference...")
del img

# 4. Перевіряємо слабке посилання
print(f"Weak link calls: {weak_link()}")

Weak link calls: <Image: 4K_Texture.png>
Ref count (approx): 1
Deleting strong reference...
Weak link calls: None


Результат: Останній прінт видасть None. Об'єкт зник, незважаючи на те, що weak_link все ще існує.

Практичний кейс: WeakValueDictionary

Ідеально для кешування.

In [5]:
# Уявіть, що це кеш завантажених користувачів
user_cache = weakref.WeakValueDictionary()

class User:
    def __init__(self, name):
        self.name = name

def get_user(name):
    u = User(name)
    user_cache[name] = u
    return u

# Користувач зайшов у систему
u1 = get_user("Alex")
print(f"Cache: {list(user_cache.keys())}") # ['Alex']

# Користувач вийшов (змінна u1 знищена)
del u1

# Кеш очистився АВТОМАТИЧНО!
# (Може знадобитися примусовий gc.collect() у деяких середовищах для демонстрації,
# але механізм працює сам)
gc.collect()
print(f"Cache after deletion: {list(user_cache.keys())}") # []

Cache: ['Alex']
Cache after deletion: []


In [6]:
import sys

small_obj = "a" * 10
print(f"Size of small: {sys.getsizeof(small_obj)} bytes. Manager: Pymalloc")

large_obj = "a" * 10000
print(f"Size of large: {sys.getsizeof(large_obj)} bytes. Manager: OS Malloc")

Size of small: 51 bytes. Manager: Pymalloc
Size of large: 10041 bytes. Manager: OS Malloc
